<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/S3Project_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [292]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [394]:
!pip install json-flatten

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [368]:
import pandas as pd
import psycopg2
import json

In [351]:
conn = psycopg2.connect(
    host='floppy.db.elephantsql.com',
    user='dnxuvido',
    password='wbCrbGaYSKPsSg-qCddu4aKGZxgAFWuz',
    database='dnxuvido'
)

In [352]:
cur = conn.cursor()

In [364]:
cur.execute("SELECT * FROM chal_test")
chal_df = pd.DataFrame(cur.fetchall())
cur.execute("SELECT * FROM grandmaster_test")
gm_df = pd.DataFrame(cur.fetchall())



In [386]:
conn.close()

In [365]:
chal_df = chal_df.rename(columns={0:'index', 1:'augments', 2:'traits', 3:'units', 4:'level', 5:'placement', 6:'cost_sum', 7:'highCost_sum', 8:'highCost_ratio'}).set_index('index').reset_index().drop(columns=['index'])
gm_df = gm_df.rename(columns={0:'index', 1:'augments', 2:'traits', 3:'units', 4:'level', 5:'placement', 6:'cost_sum', 7:'highCost_sum', 8:'highCost_ratio'}).set_index('index').reset_index().drop(columns=['index'])

In [366]:
chal_df.shape, gm_df.shape

((27840, 8), (64000, 8))

In [362]:
gm_df.head()

,augments,traits,units,level,placement,cost_sum,highCost_sum,highCost_ratio
0,"['TFT8_Augment_LeeSinCarry', 'TFT6_Augment_Por...","[{'name': 'Set8_Aegis', 'num_units': 2, 'style...","[{'character_id': 'TFT8_Galio', 'itemNames': [...",8,5,66,9,0.136364
1,"['TFT8_Augment_LeeSinSupport', 'TFT6_Augment_T...","[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Blitzcrank', 'itemName...",8,3,90,24,0.266667
2,"['TFT8_Augment_LuluSupport', 'TFT6_Augment_Lud...","[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Lulu', 'itemNames': []...",9,2,86,44,0.511628
3,"['TFT8_Augment_GalioSupport', 'TFT7_Augment_Sc...","[{'name': 'Set8_Aegis', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Galio', 'itemNames': [...",6,7,48,0,0.000000
4,"['TFT8_Augment_FioraSupport', 'TFT6_Augment_Po...","[{'name': 'Set8_Aegis', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Gangplank', 'itemNames...",7,6,69,0,0.000000


In [367]:
# Data Wrangling 용 함수
def unitCost(argument):
    switcher = {
        0: 1,
        1: 2,
        2: 3,
        3: 6,
        4: 4,
        5: 7,
        6: 5,
        7: 10,
    }
    return switcher.get(argument, -1)
  
def add_cost_sum(df):
    cols = ['cost_sum', 'high_cost_sum', 'high_cost_ratio']
    units_df = pd.DataFrame(columns = cols)
    ## 필드에 배치한 유닛 총 골드 구하기
    for i in range(0, len(df)):
        result_list = []
        tmp = pd.json_normalize(json.loads(df.iloc[i]['units'].replace("'",'"')))
        
        if len(tmp) > 0:
            cost_sum = 0
            high_cost_sum = 0
            for idx in range(0, len(tmp)):
                cost = unitCost(tmp.iloc[idx]['rarity'])
                cost_sum += cost*(3**(tmp.iloc[idx]['tier']-1))
                if cost >=4: # 4코스트 이상의 유닛에 사용한 총 골드
                    high_cost_sum += cost*(3**(tmp.iloc[idx]['tier']-1))

            high_cost_ratio = high_cost_sum / cost_sum
        else:
            cost_sum = -1
            high_cost_sum= -1
            high_cost_ratio = -1
            
        result_list += cost_sum, high_cost_sum, high_cost_ratio
        units_df = units_df.append(pd.DataFrame([result_list], columns=cols))
    units_df = units_df.reset_index(drop=True)
    df = df.merge(units_df, left_index=True, right_index=True)
    return df

In [377]:
df = chal_df.copy()
df

,augments,traits,units,level,placement,cost_sum,highCost_sum,highCost_ratio
0,"['TFT7_Augment_BandOfThieves1', 'TFT6_Augment_...","[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Annie', 'itemNames': [...",8,2,86,29,0.337209
1,"['TFT6_Augment_Battlemage1', 'TFT6_Augment_Cel...","[{'name': 'Set8_AnimaSquad', 'num_units': 1, '...","[{'character_id': 'TFT8_Kayle', 'itemNames': [...",7,4,73,4,0.054795
2,"['TFT8_Augment_ADMINTrait', 'TFT7_Augment_Prep...","[{'name': 'Set8_Admin', 'num_units': 4, 'style...","[{'character_id': 'TFT8_Sylas', 'itemNames': [...",8,6,72,12,0.166667
3,"['TFT8_Augment_StarGuardianTrait', 'TFT6_Augme...","[{'name': 'Set8_Aegis', 'num_units': 2, 'style...","[{'character_id': 'TFT8_Lux', 'itemNames': [],...",8,8,50,34,0.680000
4,"['TFT7_Augment_AFK', 'TFT6_Augment_Featherweig...","[{'name': 'Set8_Ace', 'num_units': 1, 'style':...","[{'character_id': 'TFT8_Fiora', 'itemNames': [...",8,7,74,39,0.527027
...,...,...,...,...,...,...,...,...
27835,"['TFT7_Augment_ClutteredMind', 'TFT8_Augment_V...","[{'name': 'Set8_Ace', 'num_units': 4, 'style':...","[{'character_id': 'TFT8_Galio', 'itemNames': [...",8,7,55,46,0.836364
27836,"['TFT6_Augment_SunfireBoard', 'TFT8_Augment_Yu...","[{'name': 'Set8_AnimaSquad', 'num_units': 1, '...","[{'character_id': 'TFT8_Galio', 'itemNames': [...",8,1,79,20,0.253165
27837,"['TFT8_Augment_UndergroundTheTrait', 'TFT8_Aug...","[{'name': 'Set8_Admin', 'num_units': 2, 'style...","[{'character_id': 'TFT8_Blitzcrank', 'itemName...",8,6,44,17,0.386364
27838,"['TFT6_Augment_Battlemage2', 'TFT8_Augment_Jax...","[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Blitzcrank', 'itemName...",7,5,70,12,0.171429


In [466]:
js_list = [
            {
                "augments": [
                    "TFT8_Augment_YuumiCarry",
                    "TFT6_Augment_TradeSectorPlus",
                    "TFT7_Augment_UrfsGrabBag2"
                ],
                "companion": {
                    "content_ID": "d61df27b-9446-43be-b2a9-c64c28d6db71",
                    "item_ID": 10013,
                    "skin_ID": 13,
                    "species": "PetSGCat"
                },
                "gold_left": 1,
                "last_round": 38,
                "level": 8,
                "placement": 1,
                "players_eliminated": 2,
                "puuid": "jj1df3BWdfQocT1XX90dR9v4OML_U-7qp8tnWZ7A9kqgUZ0PUBtozfxY70LVR8OZlgASKApk3H8oZA",
                "time_eliminated": 2370.826416015625,
                "total_damage_to_players": 147,
                "traits": [
                    {
                        "name": "Set8_Aegis",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_AnimaSquad",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Brawler",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Channeler",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Civilian",
                        "num_units": 1,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Duelist",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Mascot",
                        "num_units": 6,
                        "style": 3,
                        "tier_current": 3,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_OxForce",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_StarGuardian",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Supers",
                        "num_units": 3,
                        "style": 3,
                        "tier_current": 1,
                        "tier_total": 1
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Galio",
                        "itemNames": [
                            "TFT_Item_RedBuff",
                            "TFT_Item_IonicSpark",
                            "TFT_Item_FrozenHeart"
                        ],
                        "items": [
                            57,
                            36,
                            45
                        ],
                        "name": "",
                        "rarity": 0,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Nasus",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Gangplank",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_LeeSin",
                        "itemNames": [
                            "TFT_Item_RunaansHurricane",
                            "TFT_Item_RecurveBow",
                            "TFT_Item_TitanicHydra"
                        ],
                        "items": [
                            26,
                            2,
                            27
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Malphite",
                        "itemNames": [
                            "TFT_Item_ThiefsGloves",
                            "TFT_Item_Zephyr",
                            "TFT_Item_GuardianAngel"
                        ],
                        "items": [
                            99,
                            67,
                            94
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Yuumi",
                        "itemNames": [
                            "TFT_Item_JeweledGauntlet",
                            "TFT_Item_SeraphsEmbrace",
                            "TFT_Item_RabadonsDeathcap"
                        ],
                        "items": [
                            39,
                            44,
                            33
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Alistar",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 1
                    },
                    {
                        "character_id": "TFT8_Taliyah",
                        "itemNames": [
                            "TFT8_Item_MascotEmblemItem"
                        ],
                        "items": [
                            8008
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Syndra",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 6,
                        "tier": 1
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_LeeSinSupport",
                    "TFT6_Augment_TomeOfTraits1",
                    "TFT6_Augment_GachaAddict"
                ],
                "companion": {
                    "content_ID": "b890d4df-181a-43da-861d-99a72afbc602",
                    "item_ID": 38002,
                    "skin_ID": 2,
                    "species": "PetChibiYasuo"
                },
                "gold_left": 0,
                "last_round": 38,
                "level": 8,
                "placement": 2,
                "players_eliminated": 3,
                "puuid": "dWe9kb98onqv7BGaqEIW1Ulo2wyaoEdosyoEN5vJa_06mQB7-xjrsBGi9oIJE6-amoGc-Ytp59QpKg",
                "time_eliminated": 2370.826416015625,
                "total_damage_to_players": 157,
                "traits": [
                    {
                        "name": "Set8_Admin",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Aegis",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_AnimaSquad",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Brawler",
                        "num_units": 8,
                        "style": 4,
                        "tier_current": 4,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Defender",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_ExoPrime",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_SpaceCorps",
                        "num_units": 2,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Supers",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 1
                    },
                    {
                        "name": "Set8_UndergroundThe",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Blitzcrank",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Renekton",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Vi",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_LeeSin",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Riven",
                        "itemNames": [
                            "TFT_Item_WarmogsArmor",
                            "TFT_Item_IonicSpark",
                            "TFT_Item_FrozenHeart"
                        ],
                        "items": [
                            77,
                            36,
                            45
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Jax",
                        "itemNames": [
                            "TFT_Item_Bloodthirster",
                            "TFT_Item_GuinsoosRageblade",
                            "TFT_Item_TitansResolve"
                        ],
                        "items": [
                            16,
                            23,
                            25
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Sejuani",
                        "itemNames": [
                            "TFT_Item_Bloodthirster"
                        ],
                        "items": [
                            16
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Soraka",
                        "itemNames": [
                            "TFT_Item_ArchangelsStaff",
                            "TFT8_Item_BrawlerEmblemItem",
                            "TFT_Item_ArchangelsStaff"
                        ],
                        "items": [
                            34,
                            8013,
                            34
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_YuumiSupport",
                    "TFT6_Augment_TradeSectorPlus",
                    "TFT6_Augment_ItemGrabBag2"
                ],
                "companion": {
                    "content_ID": "69516294-8c8f-4ef0-a127-f4602b5217b8",
                    "item_ID": 46007,
                    "skin_ID": 7,
                    "species": "PetPupDragon"
                },
                "gold_left": 14,
                "last_round": 31,
                "level": 8,
                "placement": 8,
                "players_eliminated": 0,
                "puuid": "LqvYLBKnVs7I5WzclVcPht5QAOLo32BumQk_ZGml2PBJZJ72kYhTx9hPTCENJNjAIo-IJsy7eCk9_Q",
                "time_eliminated": 1916.803466796875,
                "total_damage_to_players": 56,
                "traits": [
                    {
                        "name": "Set8_Admin",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Aegis",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Channeler",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_GenAE",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Hacker",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 4,
                        "style": 2,
                        "tier_current": 2,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Mascot",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_OxForce",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Prankster",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_StarGuardian",
                        "num_units": 2,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_UndergroundThe",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Lulu",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Annie",
                        "itemNames": [
                            "TFT_Item_WarmogsArmor",
                            "TFT_Item_GargoyleStoneplate",
                            "TFT_Item_RedBuff"
                        ],
                        "items": [
                            77,
                            56,
                            57
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Yuumi",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Sona",
                        "itemNames": [
                            "TFT8_Item_RapidFireCannon_GenAE",
                            "TFT_Item_NeedlesslyLargeRod"
                        ],
                        "items": [
                            9013,
                            3
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Alistar",
                        "itemNames": [
                            "TFT_Item_WarmogsArmor",
                            "TFT_Item_TitanicHydra"
                        ],
                        "items": [
                            77,
                            27
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Zoe",
                        "itemNames": [
                            "TFT_Item_ThiefsGloves",
                            "TFT_Item_RunaansHurricane",
                            "TFT_Item_Shroud"
                        ],
                        "items": [
                            99,
                            26,
                            59
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Ekko",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 4,
                        "tier": 1
                    },
                    {
                        "character_id": "TFT8_Soraka",
                        "itemNames": [
                            "TFT_Item_HextechGunblade",
                            "TFT_Item_JeweledGauntlet",
                            "TFT_Item_SeraphsEmbrace"
                        ],
                        "items": [
                            13,
                            39,
                            44
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_AnnieCarry",
                    "TFT6_Augment_ComponentGrabBag",
                    "TFT6_Augment_WindfallPlusPlus"
                ],
                "companion": {
                    "content_ID": "abe6f9e9-6b26-4ff2-b09e-8977314bd230",
                    "item_ID": 19003,
                    "skin_ID": 3,
                    "species": "PetDSSquid"
                },
                "gold_left": 1,
                "last_round": 35,
                "level": 9,
                "placement": 3,
                "players_eliminated": 2,
                "puuid": "oNNTcVs8v-lO2FU22XPV4rzWYOKiniLDEK5wjLMBBSUkHXXHdDqds84EzA4leUf6LG1TDjtpfdO8EA",
                "time_eliminated": 2203.10107421875,
                "total_damage_to_players": 126,
                "traits": [
                    {
                        "name": "Set8_Admin",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Aegis",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Channeler",
                        "num_units": 6,
                        "style": 3,
                        "tier_current": 3,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Civilian",
                        "num_units": 1,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Forecaster",
                        "num_units": 1,
                        "style": 3,
                        "tier_current": 1,
                        "tier_total": 1
                    },
                    {
                        "name": "Set8_GenAE",
                        "num_units": 2,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Hacker",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Mascot",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_OxForce",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Prankster",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_StarGuardian",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_UndergroundThe",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Lux",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 1
                    },
                    {
                        "character_id": "TFT8_Annie",
                        "itemNames": [
                            "TFT_Item_GargoyleStoneplate",
                            "TFT_Item_TitanicHydra",
                            "TFT_Item_BrambleVest"
                        ],
                        "items": [
                            56,
                            27,
                            55
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Sona",
                        "itemNames": [
                            "TFT_Item_ThiefsGloves",
                            "TFT_Item_StatikkShiv",
                            "TFT_Item_SpearOfShojin"
                        ],
                        "items": [
                            99,
                            24,
                            14
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Alistar",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Leblanc",
                        "itemNames": [
                            "TFT_Item_StatikkShiv",
                            "TFT_Item_MadredsBloodrazor"
                        ],
                        "items": [
                            24,
                            12
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Ekko",
                        "itemNames": [
                            "TFT8_Item_OxForceEmblemItem"
                        ],
                        "items": [
                            8004
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Taliyah",
                        "itemNames": [
                            "TFT_Item_MadredsBloodrazor",
                            "TFT_Item_SpearOfShojin",
                            "TFT_Item_RabadonsDeathcap"
                        ],
                        "items": [
                            12,
                            14,
                            33
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Nunu",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 6,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Janna",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 6,
                        "tier": 1
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_SivirSupport",
                    "TFT6_Augment_ComponentGrabBag",
                    "TFT6_Augment_HighEndShopping"
                ],
                "companion": {
                    "content_ID": "5b42b7ca-15c7-4c79-8a7f-50cfbb70f77d",
                    "item_ID": 55004,
                    "skin_ID": 4,
                    "species": "PetKoala"
                },
                "gold_left": 0,
                "last_round": 31,
                "level": 7,
                "placement": 7,
                "players_eliminated": 0,
                "puuid": "tMSUzBQkg5dwlm0wfePhdNd9Yn7yWQ43KSQYu_kwDWb4T6EBvwmYgxe8iWqlfDh_OTHP3-L2-uuwMA",
                "time_eliminated": 1918.7159423828125,
                "total_damage_to_players": 89,
                "traits": [
                    {
                        "name": "Set8_Arsenal",
                        "num_units": 1,
                        "style": 3,
                        "tier_current": 1,
                        "tier_total": 1
                    },
                    {
                        "name": "Set8_Civilian",
                        "num_units": 1,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Deadeye",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_OxForce",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Threat",
                        "num_units": 4,
                        "style": 0,
                        "tier_current": 1,
                        "tier_total": 1
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Sivir",
                        "itemNames": [
                            "TFT_Item_GuinsoosRageblade",
                            "TFT_Item_Deathblade",
                            "TFT_Item_LastWhisper"
                        ],
                        "items": [
                            23,
                            11,
                            29
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Velkoz",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Chogath",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Chogath",
                        "itemNames": [
                            "TFT_Item_BrambleVest",
                            "TFT_Item_IonicSpark",
                            "TFT_Item_DragonsClaw"
                        ],
                        "items": [
                            55,
                            36,
                            66
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Rammus",
                        "itemNames": [
                            "TFT_Item_TitanicHydra",
                            "TFT_Item_JeweledGauntlet",
                            "TFT_Item_NegatronCloak"
                        ],
                        "items": [
                            27,
                            39,
                            6
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_AurelionSol",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Aphelios",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 6,
                        "tier": 1
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_ViSupport",
                    "TFT7_Augment_ScopedWeapons1",
                    "TFT8_Augment_DuelistEmblem2"
                ],
                "companion": {
                    "content_ID": "abff8702-0465-4a63-8d23-46a664b769be",
                    "item_ID": 6024,
                    "skin_ID": 24,
                    "species": "PetPenguKnight"
                },
                "gold_left": 0,
                "last_round": 33,
                "level": 7,
                "placement": 5,
                "players_eliminated": 0,
                "puuid": "Nwioc3CsVmejOe8ET1faI44JmKzbQbh6KeRxBMWR_Dn1lElfcfC1zv3_szZY40tD6fdqydNhulz2RA",
                "time_eliminated": 2061.921142578125,
                "total_damage_to_players": 95,
                "traits": [
                    {
                        "name": "Set8_Aegis",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_AnimaSquad",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Brawler",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Defender",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Duelist",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Prankster",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Recon",
                        "num_units": 3,
                        "style": 2,
                        "tier_current": 2,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_StarGuardian",
                        "num_units": 3,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_UndergroundThe",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Jinx",
                        "itemNames": [
                            "TFT8_Item_ReconEmblemItem"
                        ],
                        "items": [
                            8010
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Vi",
                        "itemNames": [
                            "TFT_Item_Zephyr"
                        ],
                        "items": [
                            67
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Vayne",
                        "itemNames": [
                            "TFT_Item_InfinityEdge",
                            "TFT_Item_GuardianAngel",
                            "TFT_Item_PowerGauntlet"
                        ],
                        "items": [
                            19,
                            94,
                            79
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Nilah",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Riven",
                        "itemNames": [
                            "TFT_Item_RedBuff",
                            "TFT_Item_BrambleVest",
                            "TFT_Item_FrozenHeart"
                        ],
                        "items": [
                            57,
                            55,
                            45
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Kaisa",
                        "itemNames": [
                            "TFT_Item_InfinityEdge",
                            "TFT_Item_MadredsBloodrazor",
                            "TFT8_Item_DuelistEmblemItem"
                        ],
                        "items": [
                            19,
                            12,
                            8014
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Ekko",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 4,
                        "tier": 1
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_DravenCarry",
                    "TFT6_Augment_Featherweights2",
                    "TFT6_Augment_VerdantVeil"
                ],
                "companion": {
                    "content_ID": "1b860b17-e947-4151-9456-ae38097bb6fb",
                    "item_ID": 4007,
                    "skin_ID": 7,
                    "species": "PetMiner"
                },
                "gold_left": 25,
                "last_round": 34,
                "level": 8,
                "placement": 4,
                "players_eliminated": 0,
                "puuid": "73DCPXAp5aCIijEFEU6JNIthkvlV_d7q4lUxyJT3cPKf-a-iRutBI1SoskBRotfS6o1OSjAyLCCpDw",
                "time_eliminated": 2130.628173828125,
                "total_damage_to_players": 66,
                "traits": [
                    {
                        "name": "Set8_Ace",
                        "num_units": 1,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Aegis",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Brawler",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Civilian",
                        "num_units": 1,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Defender",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Duelist",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_ExoPrime",
                        "num_units": 5,
                        "style": 3,
                        "tier_current": 2,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Mascot",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Renegade",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Supers",
                        "num_units": 3,
                        "style": 3,
                        "tier_current": 1,
                        "tier_total": 1
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Gangplank",
                        "itemNames": [
                            "TFT_Item_LastWhisper"
                        ],
                        "items": [
                            29
                        ],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_WuKong",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_Draven",
                        "itemNames": [
                            "TFT_Item_Bloodthirster",
                            "TFT_Item_TitansResolve",
                            "TFT_Item_GuardianAngel"
                        ],
                        "items": [
                            16,
                            25,
                            94
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 3
                    },
                    {
                        "character_id": "TFT8_LeeSin",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Malphite",
                        "itemNames": [
                            "TFT_Item_RedBuff",
                            "TFT_Item_LocketOfTheIronSolari"
                        ],
                        "items": [
                            57,
                            35
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Jax",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Sett",
                        "itemNames": [
                            "TFT8_Item_CivilianEmblemItem"
                        ],
                        "items": [
                            8001
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Leona",
                        "itemNames": [
                            "TFT_Item_FrozenHeart"
                        ],
                        "items": [
                            45
                        ],
                        "name": "",
                        "rarity": 6,
                        "tier": 1
                    }
                ]
            },
            {
                "augments": [
                    "TFT8_Augment_ViSupport",
                    "TFT6_Augment_SecondWind2",
                    "TFT6_Augment_RadiantRelics"
                ],
                "companion": {
                    "content_ID": "96c56568-60a4-4a0c-8d5b-cbc719e1a6e6",
                    "item_ID": 19022,
                    "skin_ID": 22,
                    "species": "PetDSSquid"
                },
                "gold_left": 2,
                "last_round": 33,
                "level": 8,
                "placement": 6,
                "players_eliminated": 0,
                "puuid": "Azxxx7DkVUfYx87uU-O-_M4FrUktnyPtr3PfU9Cfbe4Za_496GIKHcXxOFb54hdUlIDWOY0NRJl-_A",
                "time_eliminated": 2055.69091796875,
                "total_damage_to_players": 86,
                "traits": [
                    {
                        "name": "Set8_Admin",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Aegis",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_AnimaSquad",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Brawler",
                        "num_units": 6,
                        "style": 3,
                        "tier_current": 3,
                        "tier_total": 4
                    },
                    {
                        "name": "Set8_Defender",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_ExoPrime",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    },
                    {
                        "name": "Set8_Heart",
                        "num_units": 2,
                        "style": 1,
                        "tier_current": 1,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_SpaceCorps",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 3
                    },
                    {
                        "name": "Set8_Supers",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 1
                    },
                    {
                        "name": "Set8_Threat",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 1,
                        "tier_total": 1
                    },
                    {
                        "name": "Set8_UndergroundThe",
                        "num_units": 1,
                        "style": 0,
                        "tier_current": 0,
                        "tier_total": 2
                    }
                ],
                "units": [
                    {
                        "character_id": "TFT8_Blitzcrank",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 0,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Vi",
                        "itemNames": [
                            "TFT_Item_ZekesHerald",
                            "TFT_Item_ZekesHerald"
                        ],
                        "items": [
                            17,
                            17
                        ],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_LeeSin",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 1,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Riven",
                        "itemNames": [
                            "TFT_Item_Bloodthirster"
                        ],
                        "items": [
                            16
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Jax",
                        "itemNames": [
                            "TFT5_Item_DragonsClawRadiant",
                            "TFT_Item_GuinsoosRageblade",
                            "TFT_Item_Quicksilver"
                        ],
                        "items": [
                            2066,
                            23,
                            69
                        ],
                        "name": "",
                        "rarity": 2,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Sejuani",
                        "itemNames": [
                            "TFT_Item_RedBuff"
                        ],
                        "items": [
                            57
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 1
                    },
                    {
                        "character_id": "TFT8_Soraka",
                        "itemNames": [
                            "TFT_Item_ArchangelsStaff",
                            "TFT_Item_HextechGunblade",
                            "TFT_Item_JeweledGauntlet"
                        ],
                        "items": [
                            34,
                            13,
                            39
                        ],
                        "name": "",
                        "rarity": 4,
                        "tier": 2
                    },
                    {
                        "character_id": "TFT8_Urgot",
                        "itemNames": [],
                        "items": [],
                        "name": "",
                        "rarity": 6,
                        "tier": 1
                    }
                ]
            }
        ]

In [477]:
pd.json_normalize(js_list, record_path=['units', 'itemNames'], meta = ['placement', ['units', 'character_id'], ['units', 'tier'], ['units', 'rarity']]).rename(columns={0:'item_name', 'units.character_id': 'unit_name', 'units.tier': 'tier', 'units.rarity': 'rarity'})

,item_name,placement,unit_name,tier,rarity
0,TFT_Item_RedBuff,1,TFT8_Galio,3,0
1,TFT_Item_IonicSpark,1,TFT8_Galio,3,0
2,TFT_Item_FrozenHeart,1,TFT8_Galio,3,0
3,TFT_Item_RunaansHurricane,1,TFT8_LeeSin,3,1
4,TFT_Item_RecurveBow,1,TFT8_LeeSin,3,1
...,...,...,...,...,...
81,TFT_Item_Quicksilver,6,TFT8_Jax,2,2
82,TFT_Item_RedBuff,6,TFT8_Sejuani,1,4
83,TFT_Item_ArchangelsStaff,6,TFT8_Soraka,2,4
84,TFT_Item_HextechGunblade,6,TFT8_Soraka,2,4


In [435]:
pd.json_normalize(js1['units'])

,character_id,itemNames,items,name,rarity,tier
0,TFT8_Galio,"[TFT_Item_RedBuff, TFT_Item_IonicSpark, TFT_It...","[57, 36, 45]",,0,3
1,TFT8_Nasus,[],[],,0,2
2,TFT8_Gangplank,[],[],,0,3
3,TFT8_LeeSin,"[TFT_Item_RunaansHurricane, TFT_Item_RecurveBo...","[26, 2, 27]",,1,3
4,TFT8_Malphite,"[TFT_Item_ThiefsGloves, TFT_Item_Zephyr, TFT_I...","[99, 67, 94]",,1,3
5,TFT8_Yuumi,"[TFT_Item_JeweledGauntlet, TFT_Item_SeraphsEmb...","[39, 44, 33]",,1,3
6,TFT8_Alistar,[],[],,2,1
7,TFT8_Taliyah,[TFT8_Item_MascotEmblemItem],[8008],,4,2
8,TFT8_Syndra,[],[],,6,1


In [441]:
test = pd.json_normalize(js1, record_path=['units','itemNames'], meta=['placement', ['units', 'character_id'], ['units','tier']])

In [465]:
from numpy import record
pd.json_normalize(json.loads(chal_df.iloc[0]['units'].replace("'",'"')), record_path=['itemNames'], meta=['tier'])

,0,tier
0,TFT_Item_RedBuff,2
1,TFT_Item_IonicSpark,2
2,TFT_Item_Zephyr,2
3,TFT_Item_JeweledGauntlet,3
4,TFT_Item_PowerGauntlet,3
5,TFT_Item_SeraphsEmbrace,3
6,TFT_Item_Bloodthirster,2
7,TFT_Item_ThiefsGloves,2
8,TFT_Item_PowerGauntlet,2
9,TFT_Item_FrozenHeart,2


In [455]:
test = test.rename(columns={0:'item_name'})
test[['placement', 'units.character_id', 'units.tier', 'item_name']]

,placement,units.character_id,units.tier,item_name
0,1,TFT8_Galio,3,TFT_Item_RedBuff
1,1,TFT8_Galio,3,TFT_Item_IonicSpark
2,1,TFT8_Galio,3,TFT_Item_FrozenHeart
3,1,TFT8_LeeSin,3,TFT_Item_RunaansHurricane
4,1,TFT8_LeeSin,3,TFT_Item_RecurveBow
5,1,TFT8_LeeSin,3,TFT_Item_TitanicHydra
6,1,TFT8_Malphite,3,TFT_Item_ThiefsGloves
7,1,TFT8_Malphite,3,TFT_Item_Zephyr
8,1,TFT8_Malphite,3,TFT_Item_GuardianAngel
9,1,TFT8_Yuumi,3,TFT_Item_JeweledGauntlet


In [411]:
pd.json_normalize(
    json_list, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']]
)

TypeError: ignored

In [392]:
total_synergy = []
total_synergy.append(test_row_data[test_row_data['tier_current']>0]['name'])

In [393]:
total_synergy

[1           Set8_Aegis
 2       Set8_Channeler
 5           Set8_Heart
 6          Set8_Mascot
 7         Set8_OxForce
 8       Set8_Prankster
 9    Set8_StarGuardian
 Name: name, dtype: object]

In [390]:
test_row_data.sort_values('num_units', ascending=False)

,name,num_units,style,tier_current,tier_total
5,Set8_Heart,4,2,2,3
9,Set8_StarGuardian,3,1,1,4
1,Set8_Aegis,2,1,1,4
2,Set8_Channeler,2,1,1,4
3,Set8_GenAE,2,0,0,2
6,Set8_Mascot,2,1,1,4
7,Set8_OxForce,2,1,1,4
8,Set8_Prankster,2,1,1,2
0,Set8_Admin,1,0,0,3
4,Set8_Hacker,1,0,0,3


In [282]:
test_unit_df = pd.json_normalize(json.loads(df[df.placement == 2].iloc[2]['units'].replace("'",'"')))

In [283]:
# 진화등급, 아이템 착용 개수, 코스트 순서로 유닛들 정렬
test_unit_df = pd.concat([test_unit_df.drop(columns=['itemNames', 'name','items']), test_unit_df['itemNames'].apply(lambda x: pd.Series(x))], axis=1).sort_values(by=[2,'tier', 'rarity'], ascending=[False, False, False])

In [284]:
test_unit_df = test_unit_df.rename(columns={0:'item1', 1:'item2', 2:'item3'})
test_unit_df

,character_id,rarity,tier,item1,item2,item3
2,TFT8_Sona,2,3,TFT_Item_SpearOfShojin,TFT_Item_MadredsBloodrazor,TFT_Item_JeweledGauntlet
7,TFT8_Syndra,6,1,TFT_Item_ThiefsGloves,TFT_Item_RunaansHurricane,TFT_Item_ArchangelsStaff
5,TFT8_Ekko,4,2,TFT_Item_TitanicHydra,TFT4_Item_OrnnAnimaVisage,TFT8_Item_Shroud_GenAE
1,TFT8_Annie,1,3,TFT_Item_BrambleVest,TFT_Item_GargoyleStoneplate,TFT8_Item_HeartEmblemItem
6,TFT8_Soraka,4,2,TFT_Item_MadredsBloodrazor,TFT_Item_SpearOfShojin,TFT4_Item_OrnnZhonyasParadox
3,TFT8_Alistar,2,2,TFT_Item_TitanicHydra,TFT_Item_TitanicHydra,NaN
4,TFT8_Zoe,2,2,TFT_Item_SeraphsEmbrace,TFT8_Item_HeartEmblemItem,NaN
0,TFT8_Lulu,0,2,TFT4_Item_OrnnInfinityForce,NaN,NaN


In [339]:
## 재료아이템 목록
# BFSword : 대검
# TFT_Item_NeedlesslyLargeRod : 쓸큰지

## 조합아이템 목록
# TFT_Item_JeweledGauntlet : 보건
# TFT_Item_RabadonsDeathcap : 데캡
# TFT_Item_ThiefsGloves : 도장
# TFT_Item_RedBuff : 태불망
# Zephyr : 서풍
# TFT_Item_EmptyBag : ??
# TFT_Item_TitanicHydra : ??
# TFT_Item_IonicSpark : 이온충격기
# TFT_Item_Shroud : 침장
# TFT_Item_SeraphsEmbrace : 블루
# TFT_Item_InfinityEdge : 인피
# TFT_Item_RapidFireCannon : 고연포
# TFT_Item_PowerGauntlet : ??
# TFT_Item_FrozenHeart : 얼심
# TFT_Item_Bloodthirster : 피바라기
# TFT_Item_StatikkShiv : 스태틱
# TFT_Item_GuinsoosRageblade : 구인수의 격노검
# TFT_Item_BrambleVest : 가시갑옷
# TFT_Item_MadredsBloodrazor : 거학
# TFT_Item_Morellonomicon : 모렐로
# TFT_Item_SpearOfShojin : 쇼진의창
# TFT_Item_Deathblade : 죽검
# TFT_Item_LastWhisper : 라위
# TFT_Item_GargoyleStoneplate : 가고일돌갑옷
# ZekesHerald : 지크의전령
# GuardianAngl : ???
# TFT_Item_LocketOfTheIronSolari : 솔라리
# TFT_Item_RunaansHurricane : 루난의 허리케인
# TFT_Item_Quicksilver : 수은
# TFT_Item_Redemption : 구원
# TFT_Item_TitansResolve : 거인의결의
# TFT_Item_WarmogsArmor : 워모그갑옷
# TFT_Item_Chalice : 성배
# TFT_Item_DragonsClaw : 용발
# TFT_Item_ArchangelsStaff : 대천사의 지팡이

## 시너지 문양
# TFT8_Item_DuelistEmblemItem : 결투가 문양
# TFT8_Item_CivilianEmblemItem : 민간인 문양
# TFT8_Item_AegisEmblemItem : 방패대 문양
# TFT8_Item_HeartEmblemItem : 선의 문양
# TFT8_Item_ChannelerEmblemItem : 주문추적자 문양
# TFT8_Item_PranksterEmblemItem : 익살꾼 문양
# TFT8_Item_ADMINEmblemItem : 자동방어체계 문양

## 기계유망주 아이템: _GenAE

## 오른 증강체 아이템: _Ornn


In [215]:
for row in test_unit_df:
  print(row)
  # breakpoint()
  # if x["itemNames"] :
  #     print(x['character_id'], x['itemNames'])

character_id
itemNames
items
name
rarity
tier


In [37]:
total_units_table = pd.DataFrame()

In [92]:
class tft8_unit:
    def __init__(self, id, syn1, syn2, syn3 = None):
        self.id = id
        self.syn1 = syn1
        self.syn2 = syn2
        self.syn3 = syn3
    
    def whoiam(self):
        if self.syn3:
            return f"id: {self.id}, synergy: {self.syn1}, {self.syn2}, {self.syn3}"
        else:
            return f"id: {self.id}, synergy: {self.syn1}, {self.syn2}"



In [90]:
## 시너지 종류
# Set8_Admin : 자동방어체계 
# Set8_Aegis : 방패대
# Set8_Channeler : 주문추적자(spellslinger)
# Set8_StarGuardian : 별수호자
# Set8_Prankster : 익살꾼
# Set8_OxForce : 황소부대
# Set8_UndergroundThe : 지하세계(Underground)
# Set8_Mascot : 마스코트
# Set8_Hacker : 해커
# Set8_Heart : 선의
# Set8_Ace : 에이스
# Set8_Brawler : 싸움꾼
# Set8_Civilian : 민간인
# Set8_Deadeye : 특등사수(Sureshot)
# Set8_SpaceCorps : 레이저단(LaseerCorps)
# Set8_Defender : 엄호대
# Set8_ExoPrime : 메카프라임(Mecha:PRIME)
# Set8_Threat : 위협
# Set8_Recon : 정찰단
# Set8_Duelist : 결투가
# Set8_AnimaSquad : 동물특공대
# Set8_Supers : 우세
# Set8_GenAE : 기계유망주(Gadgeteen)
# Set8_Corrupted : 타락
# Set8_Forecaster : 기상캐스터
# Set8_Renegade : 무법자


In [93]:
Annie = tft8_unit('TFT8_Annie', 'Set8_GenAE', 'Set8_Oxforce', 'Set8_Channeler')
Blitzcrank = tft8_unit('TFT8_Blitzcrank', 'Set8_Admin', 'Set8_Brawler')
Camile
LeBlanc
Soraka


In [94]:
test_df["character_id"]

0    TFT8_Gangplank
1           TFT8_Vi
2       TFT8_LeeSin
3     TFT8_Malphite
4       TFT8_Ezreal
5         TFT8_Sona
Name: character_id, dtype: object